In [2]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# api requests
# Senator api request
Senator_response = requests.get(
    'https://api.propublica.org/congress/v1/116/senate/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)
# House api request
House_response = requests.get(
    'https://api.propublica.org/congress/v1/116/house/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)

#convert responses into json
senate_json_response = Senator_response.json()
house_json_response = House_response.json()


In [3]:
# Senator array
Senate = senate_json_response['results'][0]["members"]
print(len(Senate))
# House array
House = house_json_response['results'][0]["members"]    
print(len(House))

102
450


In [4]:
#Congress Array
Congress = Senate + House
print(len(Congress))

552


In [5]:

# Convert array in Dataframe
df = pd.DataFrame(Congress)

In [7]:
df = df.drop(['date_of_birth', 'twitter_account', 'facebook_account','youtube_account','url', 'rss_url','contact_form', 'in_office', 'ideal_point', 
'total_votes', 'missed_votes', 'total_present', 'last_updated', 'office', 'phone', 'fax', 'at_large', 'geoid', 'cook_pvi','dw_nominate'], axis=1)

In [8]:

df['image'] = "None"

In [9]:
# States for scraping
states = ["alabama", 'alaska',"arizona", "arkansas", "california", "colorado",'connecticut','delaware', 'florida','georgia', 'hawaii','idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri','montana', 'nebraska','nevada', 'new%20hampshire', 'new%20jersey','new%20mexico', 'new%20york', 'north%20carolina', 'north%20dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania','puerto%20rico', 'rhode%20island', 'south%20carolina', 'south%20dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west%20virginia', 'wisconsin', 'wyoming' ]

Main_url = 'https://www.congress.gov/search?searchResultViewType=expanded&q={%22congress%22:[%22116%22],%22source%22:[%22members%22],%22search%22:%22'
back_of_Main_url = "%22}"
top_images_url = 'https://congress.gov'
#reps with no photos on Congress.gov
no_photos = []
Congress_dic = {}
Congress_dic['congress_info'] = [] 

In [10]:
# Checking to see if there is a picture on the website
def CheckForImg(str):
    try:
        rep_Img = rep.find("img")['src']
        return True
    except:
        return False

In [11]:
state_abrv = {"alabama":'AL', 'alaska':'AL',"arizona":'AZ', "arkansas":'AK', "california":'CA', "colorado":'CO','connecticut':'CT','delaware':'DE', 'florida':'FL','georgia':'GA', 'hawaii':'HI','idaho':'ID', 'illinois':'IL', 'indiana':'ID', 'iowa':'IA', 'kansas':'KS', 'kentucky':'KY', 'louisiana':'LA', 'maine':'ME', 'maryland':'MD', 'massachusetts':'MA', 'michigan':"MI", 'minnesota':'MN', 'mississippi':'MS', 'missouri':'MS','montana':'MT', 'nebraska':'NE','nevada':'NV', 'new%20hampshire':'NH', 'new%20jersey':'NJ','new%20mexico':'NM', 'new%20york':'NY', 'north%20carolina':'NC', 'north%20dakota':'ND', 'ohio':'OH', 'oklahoma':'OK', 'oregon':'OR', 'pennsylvania':'PA','puerto%20rico':'PR','rhode%20island':'RI', 'south%20carolina':'SC', 'south%20dakota':'SD', 'tennessee':'TN', 'texas':'TX', 'utah':'UT', 'vermont':'VT', 'virginia':'VA', 'washington':'WA', 'west%20virginia':'WV', 'wisconsin':'WI', 'wyoming':'WY'}

In [12]:

# # for loop that is going thru every state
for state in states: 
    state_res = requests.get(Main_url+state+back_of_Main_url)
    state_soup = BeautifulSoup(state_res.text, 'html.parser')
    Rep_results = state_soup.findAll("li", {"class":"expanded"})

    for rep in Rep_results:
        photo = CheckForImg(rep)
        rep_name = rep.find("span", {"class":"result-heading"}).text.split(' ')[1:]
        rep_index = df.loc[(df['state'] == state_abrv[state]) & (df['last_name'] == rep_name[0][:-1])].index
        if photo:
            df.at[rep_index, 'image'] = rep_Img = rep.find("img")['src']

In [28]:
#exporting Dataframe to csv
df.to_csv('CongressData.csv')

In [15]:
data = df.to_dict('records')

In [16]:
type(data)

list